In [1]:
import tensorflow as tf
import numpy as np
import time

np.set_printoptions(precision=4, suppress=True)

/nfs/nhome/live/yashm/anaconda3/envs/Tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
seed=0
tf.set_random_seed(seed)

mnist = tf.keras.datasets.mnist
(x_train_data, y_train_cold),(x_test_data,y_test_cold) = mnist.load_data()
x_train_data, x_test_data = x_train_data / 255.0, x_test_data / 255.0

In [3]:
with tf.Session() as sess:
    y_train = sess.run(tf.one_hot(y_train_cold,10))
    y_test =  sess.run(tf.one_hot(y_test_cold,10))

In [4]:
x_train=[]
x_test=[]

n_train = len(x_train_data)
n_test = len(x_test_data)

for i in range(len(x_train_data)):
    x_train.append(np.ndarray.flatten(x_train_data[i]))
    
for i in range(len(x_test_data)):
    x_test.append(np.ndarray.flatten(x_test_data[i]))

In [5]:
x_train = np.reshape(x_train,[n_train,784])
x_test = np.reshape(x_test,[n_test, 784])

In [6]:
print(np.sum(y_train,0))

[5923. 6742. 5958. 6131. 5842. 5421. 5918. 6265. 5851. 5949.]


In [19]:
learning_rate=0.0005
sigma=0.0001
batch_size=400

In [20]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
y = tf.placeholder(dtype=tf.float32, shape=[None,10])

In [21]:
n_hl1 = 200
xavier = tf.contrib.layers.xavier_initializer(seed=seed)


w1 = tf.Variable(xavier(shape = ([n_hl1,784])))
b1 = tf.Variable(xavier(shape = ([n_hl1])))

w2 = tf.Variable(xavier(shape = ([10,n_hl1])))
b2 = tf.Variable(xavier(shape = ([10])))

w1_star=tf.Variable(tf.random_uniform(shape=[batch_size, n_hl1, 784], minval=1, maxval=1))
w2_star=tf.Variable(tf.random_uniform(shape=[batch_size, 10, n_hl1], minval=1, maxval=1))

In [22]:
with tf.name_scope('noise'):
    squiggle1 = tf.random_normal(shape = [batch_size,n_hl1,784], mean=0, stddev=sigma, dtype=tf.float32)
    squiggle2 = tf.random_normal(shape = [batch_size,10,n_hl1], mean=0, stddev=sigma, dtype=tf.float32)

In [23]:
with tf.name_scope('batch-prediction'):
    hl1_out = tf.nn.relu(tf.add(tf.matmul(x, tf.transpose(w1)), b1))
    pred = tf.nn.softmax(tf.add(tf.matmul(hl1_out, tf.transpose(w2)), b2))
    
    hl1_out_star=tf.nn.relu(tf.add(tf.einsum('ki,kji->kj',x,tf.add(w1_star, squiggle1)),b1))
    pred_star=tf.nn.softmax(tf.add(tf.einsum('ki,kji->kj', hl1_out_star,tf.add(w2_star, squiggle2)), b2))

In [24]:
error=tf.reduce_mean(np.square(pred-y),1)
error_star=tf.reduce_mean(np.square(pred_star-y),1)

var = sigma ** 2
k = -learning_rate * (error_star - error) / var

In [25]:
with tf.name_scope('delta'):
    delta_w1=tf.einsum('kij,k->kij',squiggle1,k)
    delta_w2=tf.einsum('kij,k->kij',squiggle2,k)
    
    delta_w1=tf.reduce_mean(delta_w1,0)
    delta_w2=tf.reduce_mean(delta_w2,0)
    
    
with tf.name_scope('update'):
    update_w1=tf.assign(w1, tf.add(w1,delta_w1))
    update_w2=tf.assign(w2, tf.add(w2,delta_w2))
    
    update_w1_star=tf.add(w1_star, delta_w1)
    update_w2_star=tf.add(w2_star, delta_w2)

In [26]:
is_correct = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = 100*tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [27]:
sess = tf.Session()
init = tf.global_variables_initializer()

sess.run(init)

In [28]:
w1_star=tf.einsum('kij, ij->kij', w1_star, w1)
w2_star=tf.einsum('kij, ij->kij', w2_star, w2)

In [17]:
# acc_summ = tf.summary.scalar('accuracy', accuracy)

# summ_dir='/nfs/nhome/live/yashm/Desktop/weight perturbation'
# test_writer=tf.summary.FileWriter(summ_dir+'/test',sess.graph)
# train_writer=tf.summary.FileWriter(summ_dir+'/train',sess.graph)

In [29]:
start = time.time()
n_epochs=20
updates= [update_w1, update_w2, update_w1_star, update_w2_star]
squiggles= [squiggle1, squiggle2]


for i in range(n_epochs):
    for j in range(0, n_train, batch_size):
        sess.run([squiggles, updates], feed_dict = {x:x_train[j:j+batch_size], y:y_train[j:j+batch_size]})
    
    if i%1==0:
#         _,test_summ=sess.run([accuracy, acc_summ],feed_dict={x:x_test,y:y_test})
#         _,train_summ=sess.run([accuracy, acc_summ],feed_dict={x:x_train,y:y_train})
#         test_writer.add_summary(test_summ, i)
#         train_writer.add_summary(train_summ, i)
        train_acc=sess.run([accuracy], feed_dict={x:x_train, y:y_train})
        print('epoch : ', i+1, 'train_acc : ', train_acc)
        sec=int(time.time()-start)
        print(int(sec/60),'m ', int(sec%60),'s')
# train_writer.flush()
# test_writer.flush()

epoch :  1 train_acc :  [9.506666]
0 m  7 s
epoch :  2 train_acc :  [16.398333]
0 m  15 s
epoch :  3 train_acc :  [12.943334]
0 m  22 s
epoch :  4 train_acc :  [10.5616665]
0 m  30 s
epoch :  5 train_acc :  [8.516667]
0 m  38 s
epoch :  6 train_acc :  [7.421666]
0 m  45 s
epoch :  7 train_acc :  [12.365]
0 m  53 s
epoch :  8 train_acc :  [13.655]
1 m  0 s
epoch :  9 train_acc :  [13.28]
1 m  8 s
epoch :  10 train_acc :  [13.781667]
1 m  15 s
epoch :  11 train_acc :  [8.665]
1 m  23 s
epoch :  12 train_acc :  [7.496667]
1 m  31 s
epoch :  13 train_acc :  [9.675]
1 m  38 s
epoch :  14 train_acc :  [9.506666]
1 m  46 s
epoch :  15 train_acc :  [10.49]
1 m  54 s
epoch :  16 train_acc :  [13.491667]
2 m  1 s
epoch :  17 train_acc :  [15.963334]
2 m  9 s
epoch :  18 train_acc :  [14.174999]
2 m  16 s
epoch :  19 train_acc :  [14.381666]
2 m  24 s
epoch :  20 train_acc :  [14.863333]
2 m  31 s


In [ ]:
 print('Test : ', sess.run([accuracy], feed_dict={x : x_test, y : y_test}),'\t')

In [ ]:
train_writer.close()
test_writer.close()